In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = False

In [8]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "cls",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [10]:
size = 5

## UMAP

In [11]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_5_cls_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_5_cls_kfold_best_model.csv


In [12]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2017604/888675771.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.691146   
                                                                         0.0005         0.688630   
                                                                         0.0010         0.660671   
                                                          (512, 512)     0.0001         0.694905   
                                                                         0.0005         0.700000   
                                                                         0.0010         0.677415   
           12              3         10        1          (256, 256)     0.0001         0.695993   
                                                                         0.0005         0.694595   
                                                                         0.0010         0.695092   
                                                          (512, 512)     0.0001         0.693166   
                                                                         0.0005         0.692482   
                                                                         0.0010         0.703883   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.656245   
                                                                         0.0005         0.653339   
                                                                         0.0010         0.584118   
                                                          (512, 512)     0.0001         0.657751   
                                                                         0.0005         0.654031   
                                                                         0.0010         0.584004   
           12              3         10        1          (256, 256)     0.0001         0.659557   
                                                                         0.0005         0.658158   
                                                                         0.0010         0.657005   
                                                          (512, 512)     0.0001         0.654349   
                                                                         0.0005         0.658667   
                                                                         0.0010         0.656664   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.659198   
                                                                         0.0005          0.656090   
                                                                         0.0010          0.616932   
                                                          (512, 512)     0.0001          0.662248   
                                                                         0.0005          0.668076   
                                                                         0.0010          0.641559   
           12              3         10        1          (256, 256)     0.0001          0.663829   
                                                                         0.0005          0.662449   
                                                                         0.0010          0.661997   
                                                          (512, 512)     0.0001          0.659824   
                                                                         0

In [13]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.697111,0.658691,"[0.7732030704815073, 0.544179523141655]",0.663988,"[0.7533315202610824, 0.5746445497630331]",0.655467,"[0.794151376146789, 0.5167820990942994]",None,0.737027,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.687605,0.657814,"[0.7587795048934945, 0.5568482284505551]",0.657303,"[0.761849710982659, 0.552755905511811]",0.658368,"[0.7557339449541285, 0.5610015982951518]",None,0.733118,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.693383,0.653409,"[0.7711145123139, 0.5357042054755856]",0.659375,"[0.7491213841578805, 0.5696278511404562]",0.650016,"[0.7944380733944955, 0.5055940330314331]",None,0.718695,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_umap_kfold_5[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001


In [15]:
best_swmhau_network_umap_kfold_5["f1"].mean()

0.6566381741261162

In [16]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.6602218203028204

In [17]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.6546168541527161

In [18]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.76769903, 0.54557732])

In [19]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.75476754, 0.5656761 ])

In [20]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.78144113, 0.52779258])

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_5,
    best_swmhau_network_grp_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_5

In [ ]:
best_swmhau_network_grp_kfold_5[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_grp_kfold_5["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_5["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_5["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_5["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_5["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_5["recall_scores"]).mean(axis=0)

# w=11

In [ ]:
size = 11

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_11

In [ ]:
best_swmhau_network_umap_kfold_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_11["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_11["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_11["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_11,
    best_swmhau_network_grp_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_11

In [ ]:
best_swmhau_network_grp_kfold_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_grp_kfold_11["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_11["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_11["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_11["recall_scores"]).mean(axis=0)

# w=20

In [ ]:
size = 20

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_20

In [ ]:
best_swmhau_network_umap_kfold_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_20,
    best_swmhau_network_grp_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_20

In [ ]:
best_swmhau_network_grp_kfold_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size = 35

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_35

In [ ]:
best_swmhau_network_umap_kfold_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_35,
    best_swmhau_network_grp_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_35

In [ ]:
best_swmhau_network_grp_kfold_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_80,
    best_swmhau_network_grp_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_cls_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_80

In [ ]:
best_swmhau_network_grp_kfold_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["recall_scores"]).mean(axis=0)